% [b1, iter] = ladlasso(y,X,lambda,...) 
% ladlasso computes the LAD-Lasso regression estimates for given complex-  
% or real-valued data.  If number of predictors, p, is larger than one, 
% then IRWLS algorithm is used, otherwise a weighted median algorithm 
% (N > 200) or elemental fits (N<200).
% INPUT: 
%   y      : numeric response N x 1 vector (real/complex)
%   X      : numeric feature  N x p matrix (real/complex)
%   lambda : non-negative penalty parameter
%   b0     : numeric optional initial start of the regression vector for 
%           IRWLS algorithm. If not given, we use LSE (when p>1).
%   intcpt : (logical) flag to indicate if intercept is in the model
%   reltol : Convergence threshold for IRWLS. Terminate when successive 
%           estimates differ in L2 norm by a rel. amount less than reltol.
% printitn : print iteration number (default = 0, no printing)
% OUTPUT:
%   b1     : (numberic) the regression coefficient vector
%   iter   : (numeric) # of iterations (only when IRWLS algorithm is used)

# Testcases

## Passed: Real Inputs, p==1, intcpt=false

y = np.asarray([1, 4 ,5 ,6 ,1])

X = np.asarray([[4,6,7,8,9]])

lambda = 4

intcpt = False

ans =

    0.6667

## Passed: Real inputs, p==1, intcpt=true

y = np.asarray([1, 4 ,5 ,6 ,1])

X = np.asarray([[4,6,7,8,9]])

lambda = 4

intcpt = true

ans =

    10
    -1

## passed Real inputs, p>1, intcpt=false
y = np.asarray([1, 4 ,5 ,6 ,1])

X = eye(5)

lambda = 4

intcpt = False

ans =

   1.0e-07 *

    0.6250
    0.6250
    0.6250
    0.6250
    0.6250
    
##  passed Real inputs, p>1, intcpt=true
y = np.asarray([1, 4 ,5 ,6 ,1])

X = eye(5)

lambda = 4

intcpt = False

ans =

    4.0000
   -0.0000
   -0.0000
    0.0000
    0.0000
   -0.0000

## complex inputs, p=1, intcpt=false

y =

   0.0000 - 1.0000i
   4.0000 + 0.0000i
   5.0000 + 0.0000i
   

X =

   6.0000 + 0.0000i
   7.0000 + 0.0000i
   0.0000 - 8.0000i
   
lambda = 3

ans =

   0.1163 + 0.0655i

In [1]:
from robustsp import *

y = [1, 4 ,5]
X = [6,7,8j]

ladlasso(y,X,3,intcpt=False)

((0.18212027363197597-0.054942298217519295j), 43, True)

In [1]:
from robustsp import *

y = np.asarray((1,4,5)) + 1j
X = np.asarray((6,7,8))
lambd = 3

y2 = np.append(y/X, 0)
X2 = np.append(np.abs(X), lambd)

ladlasso(y,X,3)

'''
ans =

   4.0000 + 1.0000i
   0.0000 + 0.0000i
   
   (array([[3.99999844e+00+1.00000000e+00j],
        [2.22222229e-07+2.16856987e-15j]]), 1999)
'''

ladlasso(y,X,3,intcpt=False)

(0.5714287679097964+0.14285757843505617j)

In [19]:
from robustsp import *
import scipy.io
import numpy as np
import pkg_resources

path = pkg_resources.resource_filename('robustsp', 'data/prostate.mat')
#def ladlassopath(y,X,intcpt=True,eps=10**-3, L= 120,reltol=1e-6,printitn=0):

X = scipy.io.loadmat(path,struct_as_record=False)['X']
y = scipy.io.loadmat(path,struct_as_record=False)['y']
intcpt = True
eps = 10**-3
L=120
reltol = 1e-7
printitn = 0
ITERMAX = 2000
n,p = X.shape

if intcpt:
    p = p+1
    medy = np.median(y) if np.isrealobj(y) else spatmed(y)
    yc = y - medy    
    lam0 = np.max(X.T @ np.sign(yc)) # max of a vector
else: np.max(X.T @ np.sign(yc))

lamgrid = eps**(np.arange(0,L+1,1)/L) * lam0 # grid of penalty values
b0 = np.concatenate((np.asarray([medy]),np.zeros(p-1)))
lambd = lamgrid[0]
##############################################
y = np.asarray(y)
X = propform(X)

N,p = X.shape

if intcpt:
    X = np.concatenate((np.ones((N,1)), X),1)

# LSE is the initial start of iteration if the initial start was not given
if b0 is None:
    b0 = np.linalg.lstsq(X[range(len(y)),:],y,rcond=None)[0]

# we use very small error tolerance value TOL between iterations.
iter = []

if printitn > 0:
    print('Computing the solution for lambda = .3f\n' % lambd);

# the case of only one predictor 
if p==1 and not intcpt: # simple linear regression and no intercept

    b1,_,_ = wmed(np.append(y[:,np.newaxis]/X, 0), np.append(np.abs(X), lambd))

    #return b1

elif p==1 and np.isrealobj(y) and N < 200  and intcpt:         
    if lambd==0:  
        b,_ = elemfits(X[:,1],y);            
    else:
        b,_ = elemfits(np.append(X[:,1],0),np.append(y,lambd));

    res = np.repeat(y[:,np.newaxis],b.shape[1],axis=1)

    res = np.sum(np.abs(res - X @ b),axis=0);
    indx = np.nanargmin(res);

    #return b[:,indx];

else:
#   use IRWLS always when p > 1 
    if printitn > 0:
        print('Starting the IRWLS algorithm..\n');

    if lambd >0:
        y = np.append(y,np.zeros(p));
        if intcpt:
             X = \
                np.vstack((X,\
                           np.hstack((np.zeros((p,1)),lambd*np.eye(p)))))
        else:
             X =  np.vstack((X,lambd*np.eye(p)))



    for i in range(ITERMAX):
        resid = np.abs(y-X@b0) 
        resid[resid<.000001]=.000001
        Xstar = X / resid[:,None]
        b1 = np.linalg.lstsq((Xstar.T  @ X),(Xstar.T @ y[:,np.newaxis]),rcond=None)[0] 
        crit = np.linalg.norm(b1-b0)/np.linalg.norm(b0);  
        if printitn !=0 and (i+1) % (printitn) == 0:
            print('ladlasso: crit(%4d) = %.9f\n' % (i,crit)) 


        if crit < reltol and i > 10:
            break 


        b0 = b1.flatten()

'''
ans = 

2,59151589437822
6,39326988051005e-11
1,59454457907121e-11
9,58008423985034e-11
2,08047969071285e-11
1,72405225298135e-11
5,27377423730947e-11
1,92688192880558e-11
1,00705477903479e-09
'''
b1


array([[2.59151589e+00],
       [6.39326988e-11],
       [1.59454459e-11],
       [9.58008439e-11],
       [2.08047969e-11],
       [1.72405225e-11],
       [5.27377423e-11],
       [1.92688194e-11],
       [1.00705481e-09]])

In [1]:
from robustsp import *
import scipy.io
import numpy as np
import pkg_resources

path = pkg_resources.resource_filename('robustsp', 'data/prostate.mat')
#def ladlassopath(y,X,intcpt=True,eps=10**-3, L= 120,reltol=1e-6,printitn=0):

X = scipy.io.loadmat(path,struct_as_record=False)['X']
y = scipy.io.loadmat(path,struct_as_record=False)['y']
intcpt = True
eps = 10**-3
L=120
reltol = 1e-7
printitn = 0
ITERMAX = 2000
n,p = X.shape

if intcpt:
    p = p+1
    medy = np.median(y) if np.isrealobj(y) else spatmed(y)
    yc = y - medy    
    lam0 = np.max(X.T @ np.sign(yc)) # max of a vector
else: np.max(X.T @ np.sign(yc))

lamgrid = eps**(np.arange(0,L+1,1)/L) * lam0 # grid of penalty values
b0 = np.concatenate((np.asarray([medy]),np.zeros(p-1)))
lambd = lamgrid[0]

ladlasso(y,X,lambd,b0,intcpt,reltol,printitn,2000)

(array([[2.59151589e+00],
        [6.39326988e-11],
        [1.59454459e-11],
        [9.58008439e-11],
        [2.08047969e-11],
        [1.72405225e-11],
        [5.27377423e-11],
        [1.92688194e-11],
        [1.00705481e-09]]), 1999)

In [ ]:
'''
[b1, iter] = ladlasso(y,X,lambda,...) 
 ladlasso computes the LAD-Lasso regression estimates for given complex-  
 or real-valued data.  If number of predictors, p, is larger than one, 
 then IRWLS algorithm is used, otherwise a weighted median algorithm 
 (N > 200) or elemental fits (N<200).
 INPUT: 
   y      : numeric response 1darray of size N  (real/complex)
   X      : numeric feature  N x p matrix (real/complex)
   lambd : non-negative penalty parameter
   b0     : numeric optional initial start of the regression vector for 
           IRWLS algorithm. If not given, we use LSE (when p>1).
   intcpt : (logical) flag to indicate if intercept is in the model
   reltol : Convergence threshold for IRWLS. Terminate when successive 
           estimates differ in L2 norm by a rel. amount less than reltol.
 printitn : print iteration number (default = 0, no printing)
 OUTPUT:
   b1     : (numberic) the regression coefficient vector
   iter   : (numeric) # of iterations (only when IRWLS algorithm is used)
'''

import numpy as np
from robustsp import *

def ladlasso(y,X,lambd,b0=None,intcpt=True,reltol=1.0e-8,printitn=0,ITERMAX = 2000):

    y = np.asarray(y)
    X = propform(X)

    N,p = X.shape

    if intcpt:
        X = np.concatenate((np.ones((N,1)), X),1)

    # LSE is the initial start of iteration if the initial start was not given
    if b0 is None:
        b0 = np.linalg.lstsq(X[range(len(y)),:],y,rcond=None)[0]

    # we use very small error tolerance value TOL between iterations.
    iter = []

    if printitn > 0:
        print('Computing the solution for lambda = .3f\n' % lambd);

    # the case of only one predictor 
    if p==1 and not intcpt: # simple linear regression and no intercept       
        b1,_,_ = wmed(np.append(y[:,np.newaxis]/X, 0), np.append(np.abs(X), lambd))        
        return b1

    elif p==1 and np.isrealobj(y) and N < 200  and intcpt:         
        if lambd==0:  
            b,_ = elemfits(X[:,1],y);            
        else:
            b,_ = elemfits(np.append(X[:,1],0),np.append(y,lambd));

        res = np.repeat(y[:,np.newaxis],b.shape[1],axis=1)

        res = np.sum(np.abs(res - X @ b),axis=0);
        indx = np.nanargmin(res);

        return b[:,indx];

    else:
    #   use IRWLS always when p > 1 
        if printitn > 0:
            print('Starting the IRWLS algorithm..\n');

        if lambd >0:
            y = np.append(y,np.zeros(p));
            if intcpt:
                 X = \
                    np.vstack((X,\
                               np.hstack((np.zeros((p,1)),lambd*np.eye(p)))))
            else:
                 X =  np.vstack((X,lambd*np.eye(p)))



        for i in range(ITERMAX):
            resid = np.abs(y-X@b0) 
            resid[resid<.000001]=.000001
            Xstar = X / resid[:,None]
            b1 = np.linalg.lstsq((Xstar.T  @ X),(Xstar.T @ y[:,np.newaxis]),rcond=None)[0] 
            crit = np.linalg.norm(b1-b0)/np.linalg.norm(b0);  
            if printitn !=0 and (i+1) % (printitn) == 0:
                print('ladlasso: crit(%4d) = %.9f\n' % (i,crit)) 


            if crit < reltol and i > 10:
                break 


            b0 = b1.flatten()

    return b1, i

In [6]:
import scipy.io
import numpy as np
from robustsp import *
import pkg_resources

path = pkg_resources.resource_filename('robustsp', 'data/prostate.mat')

Braw = scipy.io.loadmat('Braw.mat',struct_as_record=False)['Braw']
X = scipy.io.loadmat(path,struct_as_record=False)['X']
y = scipy.io.loadmat(path,struct_as_record=False)['y']
intcpt = True
eps = 10**-3
L=120
reltol = 1e-7
printitn = 0

y = np.asarray(y)
y = y if not len(y.shape)==2 else y.flatten() # ensure that y is Nx1 and not just N

X = np.asarray(X)
n,p = X.shape

if intcpt:
    p = p+1
    medy = np.median(y) if np.isrealobj(y) else spatmed(y)
    yc = y - medy    
    lam0 = np.max(X.T @ np.sign(yc)) # max of a column vector
else: np.max(X.T @ np.sign(yc))

lamgrid = eps**(np.arange(0,L+1,1)/L) * lam0 # grid of penalty values

lambd = lamgrid[1]
##################   Ladlasso   ##############################

b0 = scipy.io.loadmat('res0.mat',struct_as_record=False)['res0'].flatten()
N,p = X.shape

#if intcpt:
X = np.concatenate((np.ones((N,1)), X),1)

# if intcpt
X = np.vstack((X,np.hstack((np.zeros((p,1)),lambd*np.eye(p)))))

Xn = scipy.io.loadmat('Xn.mat',struct_as_record=False)\
['Xn']

#if lambda > 0
y = np.append(y,np.zeros(p))


'''
Soll:

i = 10

b0 =
2.5915
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
'''

resid = np.abs(y-X@b0) 
residM = scipy.io.loadmat('resid.mat',struct_as_record=False)\
['resid']
resid1 = scipy.io.loadmat('resid1.mat',struct_as_record=False)['resid1'].flatten()
resid[resid<.000001]=.000001
np.isclose(resid,resid1).all() # True

xb = scipy.io.loadmat('xb.mat',struct_as_record=False)['xb'].flatten()
np.isclose(X@b0,xb).all() # True
Xstar = X / resid[:,None] # X stimmt, resid isclose
XstarM = scipy.io.loadmat('Xstar.mat',struct_as_record=False)['Xstar']
print(np.isclose(Xstar,XstarM).all())

b1 = np.linalg.lstsq((Xstar.T  @ X),(Xstar.T @ y[:,np.newaxis]),rcond=None)[0] 

crit = np.linalg.norm(b1-b0)/np.linalg.norm(b0);
np.array_equal(X@b0,xb)

True


False

In [12]:



for i in range(2000):
    resid = np.abs(y-X@b0) 
    resid[resid<.000001]=.000001
    Xstar = X / resid[:,None]
    b1 = np.linalg.lstsq((Xstar.T  @ X),(Xstar.T @ y[:,np.newaxis]),rcond=None)[0].flatten() 
    crit = np.linalg.norm(b1-b0)/np.linalg.norm(b0);  
    if crit < reltol and i > 9:
        print('breakt at %i' % i)
        break 
    print(np.isclose(b1,Braw[:,1]).all())
    b0 = b1
i,b0

True
True
True
True
True
True
True
True
True
True
breakt at 10


(10, array([2.59151589e+00, 7.17337634e-11, 1.78910708e-11, 1.07485336e-10,
        2.33428583e-11, 1.93441837e-11, 5.91726094e-11, 2.16199676e-11,
        1.89692617e-09]))

In [11]:
import scipy.io
import numpy as np
# inputs
b0 = scipy.io.loadmat('res0.mat',struct_as_record=False)['res0'].flatten()
Xn = scipy.io.loadmat('Xn.mat',struct_as_record=False)['Xn']
yn = scipy.io.loadmat('y',struct_as_record=False)['y'].flatten()


# outputs
resid1 = scipy.io.loadmat('resid1.mat',struct_as_record=False)['resid1'].flatten() # filered resid from matlab
XstarM = scipy.io.loadmat('Xstar.mat',struct_as_record=False)['Xstar']
b1 = scipy.io.loadmat('b1.mat',struct_as_record=False)['b1'].flatten() 

# Computations
resid = np.abs(yn-Xn@b0) 
resid[resid<.000001]=.000001
np.isclose(resid,resid1).all() # True

Xstar = Xn / resid[:,None] # X stimmt, resid isclose
print(np.isclose(Xstar,XstarM).all()) # True

b1n = np.linalg.lstsq((Xstar.T  @ Xn),(Xstar.T @ yn[:,np.newaxis]),rcond=None)[0].flatten() # Todo add .flatten() 
print(np.isclose(b1,b1n).all())

True
True


array([2.59151589e+00, 7.17337634e-11, 1.78910711e-11, 1.07485338e-10,
       2.33428580e-11, 1.93441834e-11, 5.91726087e-11, 2.16199677e-11,
       1.12992783e-09])

In [32]:
# initial regression vector
binit = np.concatenate((np.asarray([medy]),np.zeros(p-1)))
currentB = ladlasso(y,X,lamgrid[0],binit,\
         intcpt,reltol,printitn)[0].flatten()
np.array_equal(currentB,Braw[:,0]) # False
all(np.isclose(currentB,Braw[:,0]))# True

1.3356497770454873e-14

In [27]:
# lamgrid stimmt, binit auch siehe Zelle 1 hoch
binit = currentB
currentB = ladlasso(y,X,lamgrid[1],binit,\
           intcpt,reltol,printitn)[0].flatten()
all(np.isclose(currentB,Braw[:,1])) # False

array([2.58754923e+00, 7.23609708e-11, 1.74807707e-11, 9.57076203e-11,
       2.40090664e-11, 1.93441844e-11, 5.90550744e-11, 2.04820759e-11,
       1.38414775e-03])